# 第9章 深層学習によって時系列を扱う仕組みを知ろう(6-10節)
ここでは、RNNとCNNによる時系列データの学習について学んでいきます。  

Google Colaboratory上で実行する場合、ランタイムがGPUになっていることを確認して下さい


In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter9

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements2.txt

In [ ]:
#audio_dataset_3classファイルが解凍されていない場合コメントアウトを外して実行して下さい
#!unzip audio_dataset_3class.zip

## 9-6 音の分類のためにデータを前処理してみよう

In [ ]:
import librosa
import pandas as pd
import numpy as np
import IPython.display as ipd

#学習データの読み込み
train_data_dir ="./audio_dataset_3class/train/"
train_df = pd.read_csv("audio_dataset_3class/train.csv", index_col=0)

#テストデータの読み込み
test_data_dir ="./audio_dataset_3class/test/"
test_df = pd.read_csv("audio_dataset_3class/test.csv", index_col=0)


In [ ]:
#学習に使うデータの音声ファイル名とラベルの1部を表示する
train_df.head()

In [ ]:
#ラベルに使われている値とその数の一覧を表示
train_df["label"].value_counts()

In [ ]:
#チェロの音声データのうちの一つを読み込む
data, rate = librosa.load(train_data_dir+ train_df[train_df["label"] == "Cello"].index[0])

#読み込んだデータを再生する
ipd.Audio(data = data, rate = rate)

In [ ]:
#読み込んだチェロの音声データの形を確認
print(data.shape)
data

In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


sampling_rate = 8000
#音の長さを3秒に切り取る
audio_duration = 3
audio_length = sampling_rate * audio_duration

#ファイル名から音声データを読み込む
def _load_files(data_dir, filenames):
  result = []
  for i, filename in enumerate(filenames):
        file_path = data_dir + filename
        data, _ = librosa.core.load(file_path, sr=sampling_rate, res_type='kaiser_fast')
        result.append(data)
  
  return result


def create_audio_dataset(train_df, test_df, train_data_dir, test_data_dir, label_dict):
    
    dim = (audio_length, 1)
    train_filenames = train_df.index
    test_filenames = test_df.index

    #学習用データとテスト用データの音声ファイルをファイル名から読み込む
    _X_train = _load_files(train_data_dir, train_filenames)
    _X_test = _load_files(test_data_dir, test_filenames)
    
    #audio_length(ここでは3秒)に音の長さを揃える
    _X_train = pad_sequences(_X_train, dtype='float32', maxlen=audio_length, padding='pre', truncating='pre', value=0.0).tolist()
    _X_test = pad_sequences(_X_test, dtype='float32', maxlen=audio_length, padding='pre', truncating='pre', value=0.0).tolist()
    
    #音のデータをStandardScalerで平均値を0、分散を１に補正する
    scaler = StandardScaler()
    scaler = scaler.fit(_X_train + _X_test)
    _X_train = scaler.transform(_X_train)
    _X_test = scaler.transform(_X_test)

    X_train = np.empty((len(train_filenames), *dim))
    for index, data in enumerate(_X_train):
      X_train[index,] = [[d] for d in data]
    
    X_test = np.empty((len(test_filenames), *dim))
    for index, data in enumerate(_X_test):
      X_test[index,] = [[d] for d in data]
    

    #以下からはlabelの作成
    labels_train = train_df["label"]
    labels_test = test_df["label"]

    y_train = np.empty(len(labels_train), dtype=int)
    for i, label in enumerate(labels_train):
        y_train[i] = label_dict[label]

    y_test = np.empty(len(labels_test), dtype=int)
    for i, label in enumerate(labels_test):
        y_test[i] = label_dict[label]
    
    #one-hot encodingする
    Y_train = to_categorical(y_train, num_classes=len(label_dict))
    Y_test = to_categorical(y_test, num_classes=len(label_dict))

    return X_train, Y_train, X_test, Y_test
    

audio_label_dict = {"Cello": 0,"Clarinet":1, "Applause":2}
X_train, Y_train, X_test, Y_test = create_audio_dataset(train_df, test_df, train_data_dir, test_data_dir, audio_label_dict)

## 9-7 LSTMで音の分類をしてみよう

In [ ]:
from tensorflow.keras.layers import Dense, LSTM, Dropout,Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

def create_lstm_model():
  input_shape = (audio_length, 1)
  
  #モデルの構築
  model_lstm = Sequential()
  model_lstm.add(LSTM(64, return_sequences=True, dropout=0.3 ,input_shape=input_shape))
  model_lstm.add(LSTM(64, return_sequences=False, dropout=0.3))
  model_lstm.add(Dense(units=len(audio_label_dict), activation="softmax"))
  model_lstm.compile(loss="categorical_crossentropy", optimizer=Adam(0.001), metrics=["acc"])
  return model_lstm

model_lstm = create_lstm_model()
#モデルの構造を表示する
model_lstm.summary()

In [ ]:
#学習開始
history = model_lstm.fit(X_train, Y_train, batch_size=16, epochs=40, validation_split=0.1, verbose=1)

In [ ]:
#学習に時間がかかるため、以下のコードのコメントアウトを外すことでモデルの重みの保存と読み込みが出来ます.
#１度保存しておき、8節以降をあとから実行する場合保存したモデルの重みを読み込むことで学習を再実行しなくて済みます

#モデルの重みの保存
#model_lstm.save_weights('./saved_models/model_lstm_weights')

In [ ]:
#保存したモデルの読み込み
#model_lstm = create_lstm_model()
#model_lstm.load_weights('./saved_models/model_lstm_weights')

## 9-8 LSTMで分類した結果を評価してみよう

In [ ]:
#予測開始
predictions = model_lstm.predict(X_test, verbose=1)
pred_labels = np.array([np.argmax(pred) for pred in predictions])
actual_labels = np.array([audio_label_dict[lab] for lab in test_df["label"]])

#正答率の算出
tmp = actual_labels == pred_labels
tmp.sum()/len(tmp)

In [ ]:
import matplotlib.pyplot as plt

#評価関数と精度のグラフ表示
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history["loss"], color="b", label="Training Loss")
ax[0].plot(history.history["val_loss"], color="g", label="Validation Loss")
ax[0].legend()

ax[1].plot(history.history["acc"], color="b", label="Training Accuracy")
ax[1].plot(history.history["val_acc"], color="g", label="Validation Accuracy")
ax[1].legend()

plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

#混同行列の作成
cf_matrix = confusion_matrix(actual_labels, pred_labels)

plt.figure(figsize=(13,13))
c = sns.heatmap(cf_matrix, annot=True, fmt="d")

#audio_label_dict = {"Cello": 0,"Clarinet":1, "Applause":2}
audio_label_list = ["Cello", "Clarinet", "Applause"]
c.set(xticklabels=audio_label_list, yticklabels=audio_label_list)
plt.plot()

## 9-9 CNNで音の分類をしてみよう

In [ ]:
from tensorflow.keras.layers import Activation, Conv1D, MaxPooling1D, GlobalMaxPool1D,Dropout

def create_cnn_model():
  #モデルの構築
  input_shape = (audio_length, 1)
  model_cnn = Sequential()    
  model_cnn.add(Conv1D(filters=128, kernel_size=9, padding='valid', input_shape=input_shape, activation='relu'))
  model_cnn.add(MaxPooling1D(pool_size=16))
  model_cnn.add(Dropout(rate=0.2))
  model_cnn.add(Conv1D(filters=64, kernel_size=3, padding='valid', activation='relu'))
  model_cnn.add(GlobalMaxPool1D())
  model_cnn.add(Dropout(rate=0.2))
  model_cnn.add(Dense(len(audio_label_dict), activation="softmax"))
  model_cnn.compile(optimizer=Adam(0.0001), loss="categorical_crossentropy", metrics=['acc'])
  return model_cnn

model_cnn = create_cnn_model()
#モデルの構造を表示する
model_cnn.summary()

In [ ]:
history = model_cnn.fit(X_train, Y_train, batch_size=16, epochs=50, validation_split=0.1, verbose=1)

## 9-10 CNNで分類した結果を評価してみよう

In [ ]:
#予測開始
predictions = model_cnn.predict(X_test, verbose=1)
pred_labels = np.array([np.argmax(pred) for pred in predictions])
actual_labels = np.array([audio_label_dict[lab] for lab in test_df["label"]])

#正答率の算出
tmp = actual_labels == pred_labels
tmp.sum()/len(tmp)

In [ ]:
#評価関数と精度のグラフ表示
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history["loss"], color="b", label="Training Loss")
ax[0].plot(history.history["val_loss"], color="g", label="Validation Loss")
ax[0].legend()

ax[1].plot(history.history["acc"], color="b", label="Training Accuracy")
ax[1].plot(history.history["val_acc"], color="g", label="Validation Accuracy")
ax[1].legend()

plt.show()

In [ ]:
#混同行列の作成
cf_matrix = confusion_matrix(actual_labels, pred_labels)

plt.figure(figsize=(13,13))
c = sns.heatmap(cf_matrix, annot=True, fmt="d")

#audio_label_dict = {"Cello": 0,"Clarinet":1, "Applause":2}
audio_label_list = ["Cello", "Clarinet", "Applause"]
c.set(xticklabels=audio_label_list, yticklabels=audio_label_list)
plt.plot()